In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.basketball-reference.com/boxscores/201606190GSW.html'
page = requests.get(url)

In [3]:
soup = BeautifulSoup(page.content)

In [13]:
tables = [x for x in soup.find_all('table') if x.get('id') in ['box-CLE-game-basic', 'box-GSW-game-basic']]

In [52]:
dfs = []
for table in tables :
    
    df = pd.read_html(str(table))[0][:-1]
    df.columns = df.columns.droplevel(0)
    df['FG'] = pd.to_numeric(df['FG'], errors = 'ignore')
    df['keep'] = df['FG'].apply(lambda x: type(x) is not str)

  #  href = [x['href']]
[type(x) for x in df['FG']]

[str, str, str, str, str, str, str, str, str, str, str, str, str, str]

In [37]:
df = pd.read_html(str(tables[0]))[0]
df.columns = df.columns.droplevel(0)
df

,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,LeBron James,46:49,9,24,.375,1,5,.200,8,10,...,1,10,11,11,2,3,5,1,27,+4
1,Kyrie Irving,43:00,10,23,.435,2,5,.400,4,4,...,3,3,6,1,1,1,2,3,26,+10
2,J.R. Smith,38:55,5,13,.385,2,8,.250,0,0,...,0,4,4,2,1,0,2,3,12,+7
3,Tristan Thompson,31:49,3,3,1.000,0,0,NaN,3,4,...,0,3,3,0,0,2,0,4,9,+2
4,Kevin Love,30:02,3,9,.333,0,3,.000,3,4,...,4,10,14,3,2,0,1,2,9,+19
5,Reserves,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
6,Richard Jefferson,25:30,1,4,.250,0,0,NaN,0,0,...,1,8,9,0,1,0,0,1,2,-8
7,Iman Shumpert,19:10,1,3,.333,1,3,.333,3,3,...,0,1,1,0,0,0,0,0,6,-9
8,Mo Williams,4:45,1,3,.333,0,1,.000,0,0,...,0,0,0,0,0,0,1,1,2,-5
9,Matthew Dellavedova,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,...,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play,Did Not Play


In [26]:
data = {
    ('A', 'B'): [1, 2, 3],
    ('C', 'D'): [4, 5, 6],
}
pd.DataFrame(data).drop(0)

,A,C
,B,D
1,2,5
2,3,6
